In [1]:
from tensorflow.keras.models import load_model
import cv2
import numpy as np

In [2]:
model = load_model('model-018.h5') #model
fc = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml') # cascade for classification

In [3]:
labels_dict = {0:'Mask', 1:'No Mask'}
color_dict = {1:(0, 0, 255), 0:(0, 255,0)} # green:mask, red:no mask

In [4]:
source = cv2.VideoCapture(0) # webcam



while (True):
    ret, img = source.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = fc.detectMultiScale(gray, 1.3,5)
    
    for x, y, w, h in faces:
        face_img = gray[y:y+h, x:x+w]
        resized = cv2.resize(face_img, (100, 100))
        normalized = resized / 255.0
        reshaped = np.reshape(normalized, (1, 100, 100, 1)) # cnn requires 4-dimensional array
        result = model.predict(reshaped)
        label = np.argmax(result, axis=1)[0]

        acc=round(np.max(result,axis=1)[0]*100,2)
        cv2.rectangle(img, (x,y), (x+w,y+h), color_dict[label], 2)
        text = "{} {}" .format(labels_dict[label], str(acc))
        cv2.putText(img,text, (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5,color_dict[label] , 2)
        

    cv2.imshow("Mask Detector", img)
    key = cv2.waitKey(1)

    if (key == ord('q')):
        break

cv2.destroyAllWindows()
source.release()




